A lot fo the things we can do in pandas, we could do using SQLAlchamy before bringing the data in, the same as we would when using SQL normally. This keeps the amount of data we are bringing in smaller and easier to work with. However, this course is time bound and I don't want to spend too long on SQL things when not everyone has access to it. Instead we'll look at some basics but do most processing in pandas. I have further notes at the two following links for more in depth SQLAlchemy including doing calculations, making more complex selections, applying functions, and joining: https://github.com/data-to-insight/ERN-sessions/blob/main/SQL%20in%20python/session_1.ipynb https://github.com/data-to-insight/ERN-sessions/blob/main/SQL%20in%20python/session_2.ipynb

I'd really recommend google, however. Also, as you'll see, if you already know SQL you can simply write the queries you want inside SQLAlchemy so you can already do the queries first, AND you don't need to learn anything.

In [7]:
# pip install sqlalchemy

from sqlalchemy import (create_engine, 
                        inspect, 
                        text, 
                        select, 
                        MetaData, 
                        Table, 
                        and_,
                        or_,
                        desc,
                        asc,
                        func,
                        )

# we need to use //// in this instance as we are giving a relative absolute path to where our DB file is
# Also remember we can't start a variable name with a number
engine_903 = create_engine("sqlite+pysqlite:////workspaces/ERN-sessions/Intermediate Python/903_database.db") #, echo=True)
# We cans et echo=True if we want to see what our DB connections are doing

connection = engine_903.connect()
inspection = inspect(engine_903)
inspection.get_table_names()

['ad1',
 'episodes',
 'header',
 'missing',
 'oc2',
 'oc3',
 'placed_for_adoption',
 'previous_permenance',
 'reviews',
 'uasc']

In [2]:
# An exercise for you, connect to, and find the table names of, the gravity db.

gravity_engine = create_engine("sqlite+pysqlite://///workspaces/ERN-sessions/SQL in python/gravity.db")
gravity_connection = gravity_engine.connect()
gravity_inspection = inspect(gravity_engine)
gravity_inspection.get_table_names()

['address',
 'address_status',
 'author',
 'book',
 'book_author',
 'book_language',
 'country',
 'cust_order',
 'customer',
 'customer_address',
 'order_history',
 'order_line',
 'order_status',
 'publisher',
 'shipping_method']

In [ ]:
# This is called 'Reflecting' and is used to reflect data from an already existing db, rather than make it,
# it takes information about the data in the table from the DB (metadata) and uses it to make tables
# to reflect a table initialise a MetaData object,
# This is a bit like making an empty table so you can put data in it later, and it'll put the right stuff in!

# https://docs.sqlalchemy.org/en/20/core/reflection.html

metadata_903 = MetaData() 

# if we print it we can see it's empty!
print(metadata_903.tables.keys())

episodes = Table('episodes', metadata_903, autoload_with=engine_903)
print(episodes.name) # prints table name
print(episodes.c.keys()) # prints column names
print(repr(episodes)) # repr() function lets us view the details of our table, like .info() for a df

dict_keys([])
episodes
['index', 'CHILD', 'DECOM', 'RNE', 'LS', 'CIN', 'PLACE', 'PLACE_PROVIDER', 'DEC', 'REC', 'REASON_PLACE_CHANGE', 'HOME_POST', 'PL_POST', 'URN']
Table('episodes', MetaData(), Column('index', INTEGER(), table=<episodes>), Column('CHILD', INTEGER(), table=<episodes>), Column('DECOM', TEXT(), table=<episodes>), Column('RNE', TEXT(), table=<episodes>), Column('LS', TEXT(), table=<episodes>), Column('CIN', TEXT(), table=<episodes>), Column('PLACE', TEXT(), table=<episodes>), Column('PLACE_PROVIDER', TEXT(), table=<episodes>), Column('DEC', TEXT(), table=<episodes>), Column('REC', TEXT(), table=<episodes>), Column('REASON_PLACE_CHANGE', TEXT(), table=<episodes>), Column('HOME_POST', TEXT(), table=<episodes>), Column('PL_POST', TEXT(), table=<episodes>), Column('URN', INTEGER(), table=<episodes>), schema=None)


In [10]:
# Now do the same for the gravity db
gravity_metadata = MetaData() 

print(gravity_metadata.tables.keys())
books = Table('book', gravity_metadata, autoload_with=gravity_engine)
print(books.name)
print(books.c.keys())
print(repr(books))

dict_keys([])
book
['book_id', 'title', 'isbn13', 'language_id', 'num_pages', 'publication_date', 'publisher_id']
Table('book', MetaData(), Column('book_id', INTEGER(), table=<book>, primary_key=True), Column('title', TEXT(), table=<book>), Column('isbn13', TEXT(), table=<book>), Column('language_id', INTEGER(), ForeignKey('book_language.language_id'), table=<book>), Column('num_pages', INTEGER(), table=<book>), Column('publication_date', DATE(), table=<book>), Column('publisher_id', INTEGER(), ForeignKey('publisher.publisher_id'), table=<book>), schema=None)


In [12]:
# In Python with is a bit like a for or an if, it does stuff in indented blocks, it lets you temporarily initialise a variable
# it's also really good for if you have a resource you don't want to use all the time (like a db connection or file stream),
# because it'll only be being used during the block

with engine_903.connect() as con:
    stmt = "SELECT * FROM episodes" # normal sql theory for querying, it's easier to do it in a Python way, as seen later
    result_proxy = con.execute(text(stmt)) # this is done to say how much data we want
    results = result_proxy.fetchall() # contains actual data
first_row = results[0]
print(first_row) # prints first row
print(first_row.CHILD) # prints CHILD column of first row

(0, 566910, '15/02/2016', 'L', 'C2', 'N3', 'T2', 'PR0', '14/08/2017', 'X1', 'CLOSE', None, None, 4244474)
566910


In [ ]:
# Find the first row of the book table, print it, then print the value of the 'title'

with gravity_engine.connect() as con:
    stmt = "SELECT * FROM book"
    result_proxy = con.execute(text(stmt))
    results = result_proxy.fetchall()
first_row = results[0]
print(first_row)
print(first_row.title) 

In [14]:
# we can also use Python to make selections, rather than SQL 
# we use the reflection of the tracks table we made earlier

with engine_903.connect() as con:
    stmt = select(episodes) # Pythonic select statement
    print(stmt) # See how it converts it to SQL
    result_proxy = con.execute(stmt) # this is done to say how much data we want
    results = result_proxy.fetchmany(size=10) # fetches first ten results
print(results)

SELECT episodes.index, episodes."CHILD", episodes."DECOM", episodes."RNE", episodes."LS", episodes."CIN", episodes."PLACE", episodes."PLACE_PROVIDER", episodes."DEC", episodes."REC", episodes."REASON_PLACE_CHANGE", episodes."HOME_POST", episodes."PL_POST", episodes."URN" 
FROM episodes
[(0, 566910, '15/02/2016', 'L', 'C2', 'N3', 'T2', 'PR0', '14/08/2017', 'X1', 'CLOSE', None, None, 4244474), (1, 566910, '14/08/2017', 'T', 'C2', 'N3', 'A3', 'PR3', None, 'X1', None, 'EG19 7TT', 'WV27 9FW', 8969628), (2, 384923, '26/04/2016', 'T', 'L1', 'N8', 'A5', 'PR2', '09/05/2017', 'X1', None, 'EH14 8EC', 'WD10 6ZQ', 1157828), (3, 384923, '09/05/2017', 'L', 'C2', 'N8', 'A5', 'PR2', None, 'X1', 'CARPL', 'EH14 8EC', 'WD10 6ZQ', 1157828), (4, 709901, '07/04/2016', 'L', 'V3', 'N8', 'T3', 'PR2', '22/01/2018', 'X1', None, 'EE28 7VZ', 'WT9 9BG', 5814849), (5, 709901, '22/01/2018', 'L', 'C1', 'N8', 'T3', 'PR2', None, 'X1', 'APPRR', 'EE28 7VZ', 'WT9 9BG', 5814849), (6, 550084, '18/02/2017', 'L', 'J1', 'N5', 'A

In [ ]:
# We can make more complex queries too, like we would using SQL or pandas.
# NOTE to differentiate these conjuenctions form the build in Python ones 
# they finish with an underscore. If they didn't, Pyhton would be trying to
# use the build in ones to a weird effect. If ever I talk about namespace, this is one reason why.

from sqlalchemy import or_
header = Table('header', metadata_903, autoload_with=engine_903)

with engine_903.connect() as con:
    # Selecting all tracks where the album ID column is 1 or 2 (Third column)
    stmt = select(header).where(or_(header.columns.ETHNIC == "REFU", 
                                      header.columns.ETHNIC == "NOBT")) 
    result_proxy = con.execute(stmt)
    results = result_proxy.fetchmany(size=20).
print(results)

[(1, 384923, 2, '09/09/2011', 'REFU', 'L034811502612', None, None), (4, 710229, 2, '15/04/2002', 'REFU', 'Z095096287583', None, None), (12, 411080, 1, '30/07/2011', 'NOBT', 'O068405655286', None, None), (17, 50224, 1, '11/01/2003', 'REFU', 'U055331591206', None, None), (40, 638861, 2, '24/09/2009', 'REFU', 'J059828899095', None, None), (71, 601515, 1, '04/07/2009', 'NOBT', 'W049855114323', None, None), (82, 803399, 2, '25/08/2007', 'REFU', 'A027843547835', None, None), (95, 995881, 2, '03/11/2004', 'REFU', 'X032007250701', None, None), (100, 291566, 2, '03/08/2008', 'REFU', 'R093718527820', None, None), (101, 456268, 2, '06/10/2004', 'NOBT', 'U092523018249', None, None), (109, 774923, 2, '15/09/2011', 'REFU', 'V005312837976', None, None), (111, 655886, 1, '12/02/2004', 'NOBT', 'I058161786961', None, None), (114, 30233, 2, '19/03/2003', 'NOBT', 'P011810180913', None, None), (120, 377841, 2, '13/06/2013', 'NOBT', 'X059426279915', None, None), (131, 690772, 1, '11/05/2009', 'REFU', 'Y0365

In [18]:
# We can easily convert results to dataframes by just passing the result to a dataframe class
import pandas as pd

ad1 = Table('ad1', metadata_903, autoload_with=engine_903)
with engine_903.connect() as con:
    stmt = select(ad1)
    result = con.execute(stmt).fetchall()
df = pd.DataFrame(result)

In [19]:
df

,index,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
0,0,988635,15/06/2013,22/11/2015,22/11/2015,1,2,MM,L11
1,1,161125,24/07/2002,08/05/2015,08/05/2015,1,1,MF,L0
2,2,590545,24/11/2008,12/09/2015,12/09/2015,1,2,F1,L11
3,3,757606,12/12/2003,16/09/2015,16/09/2015,0,1,MF,L3
4,4,921450,11/10/2006,02/03/2015,02/03/2015,0,1,FF,L3
5,5,4022,11/05/2001,11/11/2015,11/11/2015,1,1,M1,L2
6,6,127650,24/02/2013,05/09/2015,05/09/2015,0,1,F1,L2
7,7,624227,04/07/2008,27/01/2015,27/01/2015,0,2,MF,L4
8,8,711458,06/05/2009,19/03/2015,19/03/2015,0,1,F1,L4
9,9,549562,25/07/2009,30/10/2015,30/10/2015,1,1,MF,L2
